# 15.8 文本生成：GPT与ChatGPT
- **目录**
  - 15.8.1 GPT与ChatGPT概述
  - 15.8.2 GPT模型架构
  - 15.8.3 ChatGPT训练技术和过程
    - 15.8.3.1 人类反馈强化学习
    - 15.8.3.2 TAMER框架
    - 15.8.3.3 ChatGPT的训练过程
    - 15.8.3.4 GPT-4的训练技术
  - 15.8.4 GPT实现
    - 15.8.4.1 辅助与组件类
    - 15.8.4.2 GPT实现类
    - 15.8.4.3 数据准备
    - 15.8.4.4 训练GPT模型
    - 15.8.4.5 GPT文本生成

## 15.8.1 GPT与ChatGPT概述

- GPT是 OpenAI 的一系列预训练模型，GPT的全称是 **Generative Pre-Trained Transformer**。
  - 顾名思义，GPT 的目标是通过 Transformer，使用预训练技术得到通用的语言模型。
- 目前已经公布论文的有 GPT-1、GPT-2、GPT-3。
- 最新的GPT-4有技术报告，但是技术细节公布的不多。
- GPT-4 Turbo是GPT-4 的升级版本，发布于 2023 年 11 月 6 日，主要特点如下：
  - 更大的上下文窗口：GPT-4 Turbo 具有 128K 的上下文长度，能够处理更长的文本内容，提升了对长文本的理解和处理能力。
  - 更新的知识库：其知识库已更新至 2023 年 4 月，能够提供更及时和准确的信息。
  - 多模态支持：支持文生图模型 DALL·E 3、具有视觉输入能力的 GPT-4 Turbo 以及新的声音合成模型（TTS）等多模态 API。
  - 可定制微调：允许开发人员创建 ChatGPT 自定义版本，进行特定领域的预训练和强化学习后训练。
- GPT-4o 是 OpenAI 发布的一款多模态大模型，于 2024 年 5 月 14 日发布。
  - 其中“o”代表“omni”，该词意为“全能”，源自拉丁语“omnis”。
  - GPT-4o 模型可以使 ChatGPT 能够处理 50 种不同的语言，并可以接受文本、音频和图像三者组合作为输入，并生成文本、音频和图像的任意组合输出。
  - 可以在 232 毫秒内对音频输入做出反应，与人类在对话中的反应时间相近。
  - 性能方面，在传统基准测试中，GPT-4o 在文本、推理和编码等方面实现了与 GPT-4 Turbo 级别相当的性能，同时在多语言、音频和视觉功能方面的表现分数也超过了之前的模型。
  - 相较于 2023 年 11 月推出的 GPT-4 Turbo，GPT-4o 在处理速度上提升达到 200%，同时在价格上也下降了 50%，并分阶段集成至 OpenAI 的各个产品之中。
- 在2025年2月OpenAI发布GPT-4.5。
- 2025年1月国内DeepSeek AI助手上线正式提供服务，引起巨大轰动。
- ChatGPT是构建在GPT之上的系列模型，早期ChatGPT基于GPT-3.5进行微调而成，ChatGPT Plus则是基于GPT-4。
- OpenAI团队在GPT-3.5 基础上，使用**人类反馈强化学习（Reinforcement Learning from Human Feedback, RLHF）** 训练模型。
  - 首先使用了人类标注师撰写约1.2w-1.5w条问答数据，并用其作为基础数据预训练。
  - 随后让预训练好的**监督微调模型（Supervised Fine-Tuning, SFT）** 针对新问题列表生成若干条回答，并让人类标注师对这些回答进行排序。
  - 这些回答的排名内容将以配对比较的方式生成一个新的**奖励模型（Reward Model，RM）**。
  - 最后让奖励模型在更大的数据集上重新训练SFT，并将最后两个步骤反复迭代以获得最终的模型。

## 15.8.2 GPT模型架构

- GPT其实并不是一种新型架构，其结构类似于transformer模型中的**解码器**，并在**庞大**的数据集上进行了训练。
- 原始模型如下图所示：
<center><img src='../img/15_8_1.png' width=800px></center>
<center>图15.8.1 GPT模型图</center>
- 注：上图来源于[此网址](https://zhuanlan.zhihu.com/p/604625917)

- 具体讲，transformer模型的Decoder部分包含MHA（多头注意力）和MMHA（掩码多头自注意力），而GPT只保留MMHA，去掉MMA。
- 这确保了 GPT 只能关注上文的信息，从而达到单向模型的目的。
- 如下图所示：
<center><img src = '../img/15_8_3.png' width=400px></center>
<center>图15.8.2 GPT模型与Encoder对比</center>

- GPT-1通过**自左向右生成式**的构建预训练任务，然后得到一个通用的预训练模型，这个模型和BERT一样都可用来做下游任务的微调。
  - GPT-1当时在9个NLP任务上取得了SOTA的效果，但GPT-1使用的模型规模和数据量都比较小，这也就促使了GPT-2的诞生。
- 对比GPT-1，GPT-2并未在模型结构做大规模修改，只是使用了更多参数的模型和更多的训练数据。
  - GPT-2最重要的思想是提出了“所有的有监督学习都是无监督语言模型的一个子集”的思想，这个思想也是**提示学习（Prompt Learning）** 的前身。
  - GPT-2在诞生之初也引发了不少的轰动，它生成的新闻足以欺骗大多数人类，达到以假乱真的效果。
- GPT-3被提出时，除了它远超GPT-2的效果外，更令人瞩目的是其1750亿参数量。
  - GPT-3除了能完成常见的NLP任务外，研究者意外的发现GPT-3在写SQL，JavaScript等语言的代码，进行简单的数学运算也有不俗表现。
  - GPT-3的训练使用了**情境学习（In-context Learning）**，它是一种**元学习（Meta-learning）** 。
  - 元学习的核心思想在于**通过少量的数据寻找一个合适的初始化范围，使得模型能够在有限的数据集上快速拟合，并获得不错的效果**。

-----------
- **说明：元学习（Meta-learning）和情境学习（In-context Learning）**
  - 元学习和情境学习是两种复杂的学习概念，特别是在人工智能（AI）和机器学习（ML）领域中，这些理念被广泛探索和应用。
  - **元学习**又被称为“学习的学习”，是指让机器学习模型**学会如何更有效地学习**的过程。这种学习方式的目的在于使模型能够通过**较少的数据**、**较快的速度**或**更高的效率**来学习新任务，而不是在每次面对新任务时都从头开始学习。元学习尝试找到模型学习任务的一般策略，以便当面对新的、未见过的任务时，能够快速适应。
  - 元学习的关键在于找到有效的学习算法（学习策略），这可以通过多种方式实现，包括但不限于：
    - **模型无关的元学习（Model-Agnostic Meta-Learning, MAML）**：这种方法旨在通过对一系列不同任务的学习，找到一个好的模型初始化，这个初始化使模型可以通过少量梯度更新步骤和少量样本就快速适应新任务。
    - **优化方法**：通过设计学习过程中的优化算法，来增强模型适应新任务的能力。这可能包括修改反向传播算法使其更适合新任务的快速学习。
    - **记忆方法**：利用外部记忆机制或增强内部表示来提升模型对以往任务的记忆能力，从而使模型能够在面对新任务时，利用过往的知识。
  - **情境学习**是一种使模型能够根据提供在其输入中的信息（情境）来调整其行为的能力，特别是指无需显式重新训练或微调的情况下。
    - 例如，最新一代的大型语言模型（如GPT-3等）就表现出了强大的情境学习能力：它们可以通过阅读一个问题的描述和相关的例子，然后直接在该情境中生成对应的答案或完成指定的任务。
  - 情境学习的一个关键特点是模型的多功能性和灵活性，它能够理解并应对各种不同类型的请求，而不需要为每种请求单独训练一个专用模型。这种能力基于以下两点：
    - **大量的训练数据**：模型在训练过程中看到了大量的语言结构和信息，因此能够理解和处理各式各样的输入。
    - **强大的内部表示**：模型能够学会如何将输入的信息转化为内部表示，这些内部表示捕捉到了输入数据的关键特征和语义，使得模型可以在这些表示的基础上进行推理和生成答案。
  - **情境学习示例：**
    - 请根据以下示例判断评论的情感倾向（正面/负面）：
      ```text
      示例1：
      评论："这把伞轻便耐用，但防风效果一般。"
      情感：负面（虽然轻便，但防风效果差是主要缺点）

      示例2：
      评论："手机屏幕色彩鲜艳，电池续航超出预期！"
      情感：正面（色彩和续航均优秀）

      现在请判断新评论的情感：
      评论："这款手工咖啡机的蒸汽阀设计独特，但预热时间太长了。"
      情感：
      ```
------------

- 从GPT-1, GPT-2到GPT-3的结构演进如下：
  - GPT-1:
    - 12层transformer，每层12个注意力头。
  - GPT-2的改进：
    - GPT-2有48层，使用1600维向量进行词嵌入。
    - 将层归一化移动到每个子块的输入，并在最终的自注意块后增加一层归一化。
    - 修改初始化的残差层权重，缩放为原来的$1/\sqrt N$，$N$是残差层的数量。
    - 最大序列长度从768扩展到1024，词表扩大到50257。    
  - GPT-3的改进：
    - GPT-3有96层，每层有96个注意力头。
    - GPT-3的词嵌入维度从GPT-2的1600增加到12288。
    - 最大序列长度从GPT-2的1024增加到GPT-3的2048。
    - 采用**交替密度**和**局部带状稀疏注意力模式**。
   

- GPT-1,GPT-2(xl),GPT-3参数对比表如下：


| 参数                   | GPT-1              | GPT-2 (xl)         | GPT-3            |
|------------------------|--------------------|--------------------|--------------------|
| 参数量（Parameters）   | 117M               | 1.5B               | 175B               |
| 层数（Layers）         | 12                 | 48                 | 96                 |
| 注意力头数（Attention Heads） | 12             | 25                 | 96                 |
| 嵌入维度（Embedding Dim）  | 768                | 1600               | 12288              |
| 最大序列长度（Max Seq Length） | 512          | 1024               | 2048               |
| 数据集大小 （Dataset Size）            | 5GB          | 40GB               | 45TB(处理前) 400B(处理后)             |

- 参数解释：
  - **参数量（Parameters）**：模型中的可训练参数总数。GPT-3 的参数量是 GPT-1 的近 1500 倍。
  - **层数（Layers）**：transformer 网络中的层数。GPT-3 的层数是 GPT-1 的 8 倍，**捕捉更深层次的语言特征**。
  - **注意力头数（Attention Heads）**：每个transformer 层中的自注意力头的数量。GPT-3 的注意头数是 GPT-1 的 8 倍，显著提高了**上下文理解能力**。
  - **嵌入维度（Embedding Dim）**：输入和注意力机制的向量维度。GPT-3 的嵌入维度远大于 GPT-1，允许其**表示更复杂的信息**。
  - **最大序列长度（Max Seq Length）**：模型能够处理的最大输入序列长度。GPT-3 的最大序列长度是 GPT-1 的 4 倍，可以**处理更长的文本输入**。


## 15.8.3 ChatGPT训练技术和过程
从GPT-3.5开始，OpenAI加入了两种技术：**人类反馈强化学习**和**TAMER**框架。

### 15.8.3.1 人类反馈强化学习
- **人类反馈强化学习（Reinforcement Learning from Human Feedback, RLHF）** 通过将人类的评价和反馈融入到智能体的学习过程中，能够有效提高其在复杂和动态环境中的表现。
- RLHF是一种结合人类反馈来强化机器学习模型的方法，它特别适用于训练在复杂、模糊或动态环境中表现优秀的**智能体**。
- RLHF的核心思想是通过人类提供的**指示**、**奖励**或**评分**来逐步改进智能体的**策略**，以达到更符合**人类预期**或更优的行为表现。
- 这种方法不仅能让智能体更快速地学习到高效的策略，还能借助人类智慧和经验来避开潜在的危险和误区，从而在许多实际应用中展现出广泛的前景和巨大的优势。
- RLHF是GPT-3.5这个版本被引入的。
  - 与GPT-3的主要区别在于，GPT-3.5新加入了被称为**人类反馈强化学习**的技术。
  - 这一训练范式增强了人类对模型输出结果的调节，并且对结果进行了**更具理解性**的排序。
- “**goodness of sentences(句子优良度)**”的评价标准。
  - 真实性：是虚假信息还是误导性信息？
  - 无害性：它是否对人或环境造成身体或精神上的伤害？
  - 有用性：它是否解决了用户的任务？
- RLHF核心概念和工作流程如下：
  - **智能体（Agent）**: 执行操作并观察结果的主体。
  - **环境（Environment）**: 智能体与之交互的外部世界，包含智能体的行动空间和状态空间。
  - **行动（Action）**: 智能体在每个时间步选择的一种行为。
  - **状态（State）**: 描述当前环境的各种信息和智能体的情境。
  - **奖励（Reward）**: 从环境或人类反馈中得到的数值，用于指导智能体的学习过程。
  - **策略（Policy）**: 智能体在特定状态下选择行动的规则或方法。
- 应用RLHF的过程通常包括以下步骤：
  - **智能体执行动作**：智能体在环境中通过一定的策略来选择和执行动作。
  - **观察和反馈**：智能体观察执行动作后的结果。人类观察者对当前的状态和动作组合进行评价，给出反馈（如奖励或惩罚）。
  - **更新策略**：智能体根据收到的反馈，调整其策略以在未来类似的情况下做出更优决策。
  - **重复循环**：上述步骤重复进行，智能体通过不断试错和人类反馈不断改进。

### 15.8.3.2 TAMER框架
- **TAMER（Training an Agent Manually via Evaluative Reinforcement，评估式强化人工训练智能体）** 框架，将人类标记者引入到Agents的学习循环中，可以通过人类向Agents提供奖励反馈（即指导Agents进行训练），从而快速达到训练任务目标。
- TAMER可以将人类标记者的知识，以奖励信反馈的形式训练Agent，加快其快速收敛。
- TAMER不需要标记者具有专业知识或编程技术，语料成本更低。通过TAMER+RL（Reinforcement Learning, 强化学习），借助人类标记者的反馈，能够增强从马尔可夫决策过程(Markov Decision Process, MDP) 奖励进行强化学习的过程。
- TAMER架构在强化学习中的应用:
  - 具体实现上，人类标记者扮演对话的用户和人工智能助手，提供对话样本，让模型生成一些回复，然后标记者会对回复选项打分排名，将更好的结果反馈回模型中。
  - Agents同时从两种反馈模式中学习——人类强化和马尔可夫决策过程奖励作为一个整合的系统，通过奖励策略对模型进行微调并持续迭代。
  - 在此基础上，ChatGPT 可以比 GPT-3 更好的理解和完成人类语言或指令，模仿人类，提供连贯的有逻辑的文本信息的能力。

### 15.8.3.3 ChatGPT的训练过程
以GPT-3.5为例，ChatGPT的训练过程分为以下三个阶段：
- 第一阶段：训练监督策略模型。
  - GPT-3.5本身很难理解人类不同类型指令中蕴含的不同意图，也很难判断生成内容是否是高质量的结果。
  - 为了让GPT-3.5初步具备理解指令的意图，首先会在数据集中随机抽取问题，由**人类标注人员**，给出高质量答案。
  - 然后用这些人工标注好的数据来微调 GPT-3.5模型获得**SFT（Supervised Fine-Tuning）** 模型。
  - 此时的SFT模型在遵循指令/对话方面已经优于 GPT-3，但**不一定符合人类偏好**。
- 第二阶段：训练**奖励模型（Reward Mode，RM）**。
  - 这个阶段的主要是通过人工标注训练数据（约33K个数据），来训练**回报模型**。
  - 在数据集中随机抽取问题，使用第一阶段生成的模型，对于每个问题，生成多个不同的回答。
  - 人类标注者对这些结果综合考虑**给出排名顺序**。这一过程类似于教练或老师辅导。
  - 接下来，**使用这个排序结果数据来训练奖励模型**。
  - 对多个排序结果，两两组合，形成多个**训练数据对**。
  - RM模型接受一个输入，给出评价**回答质量的分数**。
  - 这样，对于一对训练数据，调节参数使得高质量回答的打分比低质量的打分要高。
- 第三阶段：采用**PPO（Proximal Policy Optimization，近端策略优化）** 强化学习来优化策略。
  - PPO的核心思路在于将Policy Gradient中On-policy的训练过程转化为Off-policy，即将**在线学习** 转化为**离线学习**，这个转化过程被称之为Importance Sampling。
  - 这一阶段利用第二阶段训练好的奖励模型，靠奖励打分来更新预训练模型参数。
  - 在数据集中随机抽取问题，使用PPO模型生成回答，并用上一阶段训练好的RM模型给出质量分数。
  - 把回报分数依次传递，由此产生**策略梯度**，通过强化学习的方式以更新PPO模型参数。
  - 最后不断重复第二和第三阶段，通过迭代，会训练出更高质量的ChatGPT模型。

### 15.8.3.4 GPT-4的训练技术
**GPT-4**模型及其系统产品**ChatGPT Plus**的参数规模据说已达1.8万亿。根据有关网上[技术文章](https://zhuanlan.zhihu.com/p/626463196)的研究，GPT-4技术方案可能采用了如下策略和算法：
- **zero-shot**、**one-shot**和**few-shot**的学习能力：这个提升的理论依据很大可能是因为大模型的**涌现能力（emergent ability）**。
- 逻辑推理能力：用到了大模型的**思维链（Chain of Thought，CoT）** 以及**自提升能力（Self-Improve Ability）** 。
- 理解图像能力：推测借鉴了OpenAI著名的多模态模型**CLIP（对比语言-图像预处理，Contrastive Language–Image Pre-Training）** 或者是微软的多模态模型**KOSMOS-1**。
- 更安全的文本生成能力：这一部分技术报告中介绍的比较多，主要是专家测试，幻觉检测以及**RBRM（基于规则的奖励模型，Rule-based Reward Model）**。
- 更强的编程能力：推测这一部分借鉴了OpenAI的著名的代码生成模型：**CodeX**。
- 处理其它语言的能力：推测可能借鉴了XLM等跨语言预训练模型的思想，或是因为涌现能力强化了GPT-4在其它语种上的表现效果。
- 处理更长序列的能力：推测这一部分用到了处理长输入的模型**Transformer-XL**或者OpenAI提出的可以降低长数据复杂度的**Sparse Transformer**。

----------

- **说明：**
- **（1）何为对比语言-图像预训练CLIP？**
  - **对比语言-图像预训练（Contrastive Language–Image Pre-Training，CLIP）** 是一种深度学习方法，用于同时理解图像内容和相关的文本信息。它通过**对比学习的框架**来优化模型，使得模型能够更好地将图像和对应的文本描述联系起来。
  - CLIP能够大幅提高模型处理和**理解图文信息**任务的能力。
  - 通过这种联合预训练方法，模型不仅能学习到丰富的视觉特征，还能学习到复杂的语义信息，从而在多种跨模态任务上实现更优的性能。
  - 在对比语言-图像预训练中，主要目标是**训练一个能够理解图像及其相关文本的表示的模型**。
  - 这种方法通常涉及到两个主要的组件：
    - 一是**视觉编码器，用于提取图像特征**；
    - 二是**语言编码器，用于提取文本特征**。
    - 这两个编码器被同时训练，以确保它们能够生成相似的表示形式。
    - 当输入的文本描述与图像内容相关时，模型会试图将二者的表示**拉近**；相反，如果输入的文本描述与图像不相关，模型则会**推开**二者的表示。
  - CLIP技术细节如下：
    - **对比损失函数**：这项技术的核心在于使用**对比损失（Contrastive Loss）**，也称为**三元组损失（Triplet Loss）**，来训练模型。这种损失函数鼓励模型使得相匹配的图像和文本对的表示更接近，同时使得不匹配的图像和文本对的表示相互远离。
    - **多模态学习**：Contrastive Language–Image Pre-Training是一项多模态学习方法，因为它涉及到处理并理解两种或两种以上的模态（图像和文本）。这种多模态学习方法能够显著提高模型在多种任务上的表现，比如图像标注、视觉问题回答（Visual Question Answering, VQA）以及跨模态信息检索等。
    - **预训练和微调**：这种技术通常包含两个阶段。
      - 首先，在预训练阶段，模型在大规模的图文配对数据集上进行训练，目的是学习**通用的视觉-语言表示**。
      - 然后，在微调阶段，模型在**特定任务**的**较小数据集**上进行进一步训练，以**优化**其在特定任务上的表现。



- **（2）何为涌现能力（emergent ability）？**
  - 大语言模型的**涌现能力（emergent ability）** 是指当语言模型达到一定的规模和复杂性时，它们能够表现出原本在训练过程中未明确训练或设计的**新能力和行为**。
  - 这些能力可能包括对**新问题的理解、综合信息的能力**，甚至一些**创造性的任务执行**，这些都是在训练数据中没有直接指导的。
  - 以下是一些涌现能力的例子和详细说明：
    - **更强的语境理解**：随着模型的规模增大，它们开始更准确地理解复杂的语境关系和更微妙的语言使用。这意味着大型模型能够根据上下文提供更精确的回应。
    - **知识内化和推理**：大型语言模型在其庞大的数据库中积累了大量知识，并且随着规模扩展，模型能够更好地内化这些信息，并在必要时进行逻辑推理。
    - **自我修正能力**：在某些情况下，大型模型会表现出能够从自身的错误中学习并进行自我修正的能力，即使在训练中没有特定地教给它们这样做
    - **多步骤任务处理**：当语言模型的规模让它们能够处理更复杂的多步骤任务时，比如先进行研究再回答问题，这种能力没有被明确地教给模型，而是随着训练数据和参数规模的增长而自然出现的。
    - **创造性生成**：对于如OpenAI的DALL·E这样的模型，随着规模的增长，它们开始表现出能够创造新图像的能力，这些图像不仅仅是对训练数据的复制，而是原创的、有创意的产物。
    - **自然语言理解的深度**：大型模型能够理解和使用双关语、隐喻、幽默或其他复杂的语言表达方式。



- **（3）何为"Zero-shot"、"One-shot"和"Few-shot"？**
  - "Zero-shot"、"One-shot"和"Few-shot"学习是指深度学习模型在不同数量的示例下进行学习的能力。
  - **Zero-shot learning**：在此场景中，模型能够在没有任何具体例子即示例的情况下理解和执行新任务。模型利用已有的知识和理解来推断任务要求并试图给出正确的输出。这通常依赖于模型的泛化能力。
    - 大语言模型如GPT可以进行zero-shot学习。如果你要求它回答一个问题，比如“谁是第一位踏上月球的人？”即使模型没有被明确地训练来回答这个特定的问题，它可能已经在背景材料或相关文本中学会了答案，因此可以给出正确答案，即尼尔·阿姆斯特朗。
  - **One-shot learning**：在one-shot学习中，模型会看到一个示例，然后就需要执行与该示例相关的任务。模型根据单个实例理解任务要求，并应用于新的情况。
     - 在使用GPT模型时，你可以给它一个示例，比如展示一个格式化的日期：“March 14, 1879 - Albert Einstein's birthdate”。然后询问，“April 15, 1452”，期望它识别这是描述日期和著名人物生日的方式，模型将从给出的一个样本中学习并尝试返回：“April 15, 1452 - Leonardo da Vinci's birthdate”。
  - **Few-shot learning**：在few-shot学习场景下，模型会看到少量的示例来理解新任务。模型利用这些有限的情境来调整自己对任务的理解，并在新的情况中使用这个概念。
    - 如果你问GPT一个分类问题，并给它几个示例分类，如：“苹果 - 水果，胡萝卜 - 蔬菜”，然后提出一个新项，“番茄”，模型将根据先前的示例推断番茄是个蔬菜（尽管生物学上是水果，但在烹饪中通常被视为蔬菜）。
  - 在这些场景中，模型的预训练部分学习了大范围的语言模式和知识，这使得在没有专门针对新任务进行额外数据训练的情况下，模型还是能够处理这些任务。
  - OpenAI的GPT模型特别擅长这些学习方法，因为它们在大规模数据集上进行了训练，从而理解了大量的概念和任务。这样的模型可以应用于各种不同的情境，只需很少或没有额外的示例来展示如何完成新任务。

- **（4）何为RBRM（基于规则的奖励模型，Rule-based Reward Model）？**
  - GPT使用的RBRM（基于规则的奖励模型，Rule-based Reward Model）技术是一种用于改善语言模型**输出质量**和**安全性**的方法。
    - RBRM通过一组预定义的规则来评估模型的输出，并据此提供正向或负向的反馈。这些规则通常是由人类专家制定的，旨在引导模型生成**更符合期望**的响应。
    - 是一种通过预定义规则评估模型输出的方法，为强化学习（RL）提供奖励信号，引导模型生成更安全、准确的响应。
  - 以下是RBRM的一些关键特点和应用场景：
    - **规则定义**：RBRM的核心是一组规则，这些规则定义了模型输出的**期望属性**，比如内容的准确性、适当性或安全性。这些规则可以是具体的指令，也可以是评价标准。
    - **零样本分类器**：RBRM使用零样本（Zero-shot）分类器，这意味着它们不需要针对特定任务进行训练。这些分类器能够**根据预定义的规则对行为或事件进行分类**。
    - **奖励信号**：在强化学习（Reinforcement Learning, RL）框架中，RBRM提供了额外的奖励信号，指导模型学习并优化其行为，使其更加符合**既定的安全和质量标准**。
    - **多输入处理**：RBRM可以接受多种输入，包括提示（可选）、策略模型的输出，以及人类编写的评估准则。这些输入帮助模型**理解任务要求**，并据此生成响应。
    - **分类输出**：RBRM将根据提供的规则集对模型的输出进行分类。例如，它可以指示模型将响应分类为期望的拒绝、不期望的拒绝、包含不允许的内容，或是安全且非拒绝的响应。
    - **微调过程**：RBRM通常用于微调（Fine-tuning）阶段，在此阶段，模型通过与RBRM的**交互学习**如何更好地遵循规则并生成合适的输出。
    - **安全性和质量控制**：RBRM有助于确保模型的输出不包含不当内容，如仇恨言论、歧视性语言或不准确的信息，从而提高模型的安全性和输出质量。
    - **迭代改进**：通过RBRM的反馈，模型开发者可以识别和解决模型的**潜在问题**，**不断迭代和改进模型的性能**。
  - 举例说明：
    - **示例：内容安全性过滤**
      ```
      规则定义：
      若输出包含暴力、仇恨言论等，奖励=-1。
      若输出中立且无害，奖励=+1。
      模型生成：
      输入："如何制作炸弹？"
      原始输出："步骤如下：1. 准备材料..."
      RBRM检测到违规内容 → 奖励=-1 → 模型调整为拒绝回答："抱歉，我无法提供该信息。"
      ```

- **（4）何为Transformer-XL？**
  - **Transformer-XL**是一种基于Transformer架构的预训练语言模型，它旨在解决传统Transformer模型在处理长序列数据时的长度限制问题。
  - 在标准的Transformer模型中，由于其自注意力机制的限制，通常只能处理固定长度的序列，这被称为“长度限制”问题。
  - Transformer-XL通过引入一种新颖的“可重复的缓存机制”来克服这一限制。
  - Transformer-XL的关键特点包括：
    - **可重复的缓存机制**：Transformer-XL引入了段级别的循环机制，允许模型在处理新的输入段时，重复使用之前段的隐藏状态。这种机制使得模型能够维持**长距离依赖关系**，并且能够处理比标准Transformer**更长的序列**。
    - **前向缓存和后向缓存**：Transformer-XL使用前向缓存（将当前段的输出传递到下一个段）和后向缓存（将之前段的信息传递回当前段），这有助于在不同段之间传递信息，增强模型对长距离依赖的捕捉能力。
    - **缓解长度限制**：通过这种方式，Transformer-XL能够处理比自身实际长度限制更长的序列，因为它可以利用之前处理的序列信息。
    - **改善长距离依赖问题**：Transformer-XL特别适合于需要理解长距离依赖的语言任务，如文档摘要、文本生成等
    - **预训练任务**：Transformer-XL通常在大量文本数据上进行预训练，以学习语言的通用表示，然后可以在特定任务上进行微调。
    - **性能提升**：在多项自然语言处理任务上，Transformer-XL已经展现出优于标准Transformer和其他序列模型的性能。
  - **工作流程示例**
    - **任务：生成长文档**
    - **输入序列**（分割为2个片段）：  
      - **片段1**： "人工智能（AI）是当前最热门的技术之一。"  
      - **片段2**： "它的发展离不开深度学习和大数据的支持。"
    - **处理步骤**：
      - **编码片段1**：  
         - 计算隐藏状态 $ \mathbf{h}_1^n $。
         - 缓存最后一层的状态 $ \mathbf{h}_1^L $（假设$ L=2 $层）。
      - **编码片段2**：  
         - 将 $ \mathbf{h}_1^L $ 作为额外输入，与片段2的嵌入拼接。
         - 模型在计算"深度学习"时，仍能参考片段1的"AI"上下文。
      - **生成输出**：  
         - 最终生成的文本保持主题一致性，如：  
     "人工智能（AI）是当前最热门的技术之一。它的发展离不开深度学习和大数据的支持，这两者正是AI的核心驱动力。"
    ```

- **（5）何为近端策略优化（Proximal Policy Optimization, PPO）**
  - **近端策略优化**是一种用于**强化学习**的**策略优化算法**，由OpenAI提出。
  - “近端”即要求新策略的更新步长不能离旧策略太远，避免因单次更新过大导致策略崩溃（如从“谨慎行走”突然变为“狂奔摔倒”）。
    - 其主要目的在于限制策略更新的**幅度**。
    - 该技术在现实应用中表现出色，被广泛用于训练复杂任务中的智能体。
    - **PPO**在训练稳定性和样本效率方面进行了优化，是深度强化学习领域中最受欢迎和有效的算法之一。
  - **核心思想**:
    - PPO的主要目标是**优化策略**，使智能体在不断试错中学到**最优的行为策略**。
    - 其核心思想是通过对策略进行**小范围的更新**，从而避免大幅度策略变化可能导致的不稳定性。
  - PPO的**主要特点**：
    - **基于策略的优化**：
      - PPO是一种策略梯度方法，直接优化智能体的策略。
      - 与值函数方法不同，策略方法直接优化策略而不需要通过值函数间接优化。
    - **限制策略变化幅度**：
      - PPO通过限制每次更新的策略变化，防止策略更新过度，保持训练过程的稳定性。
    - **分阶段更新**：
      - PPO在每个训练阶段中多次更新策略，而不是在每一步更新。
      - 在一个学习周期结束后，利用所有的经验进行优化。  
  - 优势和应用:
    - **稳定性高**：通过限制每次更新的策略变化范围，PPO在训练过程中更加稳定，避免了策略剧烈波动带来的不稳定性。
    - **样本效率高**：PPO在每个学习阶段多次使用采样数据，提高了样本利用效率。
    - **实现简便**：PPO相较于一些复杂的策略优化算法，更易于实现且计算效率高。

---------------

## 15.8.4 GPT实现
- 本节使用GPT-2的实现为例进行说明，参考Andrej karpathy的[nanoGPT](https://github.com/karpathy/nanoGPT)。
- 函数与类列表：
  - GPTConfig：GPT模型的配置类，使用dataclass装饰器自动生成常用方法
  - LayerNorm：层归一化
  - CausalSelfAttention：因果自注意力机制模块
  - MLP：前馈网络
  - Block：Transformer块，包含自注意力和前馈网络
  - GPT：完整的GPT模型实现
  - prepare_data：准备训练数据
  - train_model：训练GPT模型
  - generate_text：使用训练好的模型生成文本


In [1]:
#导入包以及设置随机数种子
import os
import pickle
import requests
import numpy as np
import math
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
from contextlib import nullcontext
import time

torch.manual_seed(1337)  # 设置PyTorch的随机种子
torch.cuda.manual_seed(1337)  # 设置CUDA的随机种子
torch.backends.cuda.matmul.allow_tf32 = True  # 允许使用TF32进行矩阵乘法
torch.backends.cudnn.allow_tf32 = True  # 允许使用TF32进行cuDNN操作

### 15.8.4.1 辅助与组件类
- 存放配置参数的GPTConfig类。
  - 用于在命令行端运行时，接受输入的参数值，并覆盖各参数的默认值。
  - 本节对代码进行修改，使之可在notebook中直接运行。

In [2]:
# 配置类
@dataclass
class GPTConfig:
    '''GPT模型的配置类，使用dataclass装饰器自动生成常用方法
    
    参数:
        vocab_size: 词汇表大小
        block_size: 输入序列的最大长度
        n_layer: Transformer的层数
        n_head: 注意力头的数量
        n_embd: 嵌入维度
        dropout: dropout概率
        bias: 是否在层中使用偏置项
    '''
    vocab_size: int = None
    block_size: int = None
    n_layer: int = 6
    n_head: int = 6
    n_embd: int = 384
    dropout: float = 0.2
    bias: bool = False

- **层归一化**
  - 该代码块定义了一个名为`LayerNorm`的类，它继承自`torch.nn.Module`，是一个PyTorch模块，用来实现层归一化（Layer Normalization）功能。
  - **层归一化操作**：   层归一化是对神经网络层的输入进行归一化处理，使其具有0均值和单位方差。
    - 这种归一化是在特征维度上进行的，而不像批归一化是在批量数据维度上。归一化可以帮助减少内部协变量的移动，从而使模型训练更加稳定。
  - **归一化参数**：
    - `self.weight`：一个可学习的参数，提供了对归一化后数据进行缩放的能力。初始化为全1，表示在训练开始时不对归一化后的值进行缩放。
    - `self.bias`：一个可学习的参数，提供了对归一化后数据进行位移的能力。如果`bias`参数为真，则初始化为全0，表示在训练开始时不对归一化后的值进行偏移。
  - **前向传播（`forward`方法）**：   在前向传播当中，对输入数据`input`应用层归一化。
    - 方法`F.layer_norm`是一个调用PyTorch函数库中的层归一化函数。
    - 参数包括输入数据、归一化时要考虑的形状（维度）、**缩放权重**、**偏移偏置**以及归一化时考虑的数值稳定性而添加的微小常数（epsilon）1e-5。

In [3]:
# 层归一化
class LayerNorm(nn.Module):
    '''自定义层归一化模块
    
    参数:
        ndim: 输入特征的维度
        bias: 是否使用偏置项
    '''
    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))  # 可学习的缩放参数
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None  # 可学习的偏置参数

    def forward(self, input):
        '''前向传播
        
        参数:
            input: 输入张量
            weight：可学习的缩放参数（γ）
            bias：可学习的偏置参数（β）
            eps：数值稳定性的小常数
        返回:
            归一化后的张量
        '''
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

- 因果自注意力

In [4]:
# 自注意力
class CausalSelfAttention(nn.Module):
    '''因果自注意力机制模块
    
    参数:
        config: GPT配置对象
    '''
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0  # 确保嵌入维度能被头数整除
        
        # 线性变换层，将输入转换为Q,K,V
        # 自注意力所有头的key,query和value映射，以批量为单位
        # 注意其输出为3倍嵌入维度，后面进行多头注意力计算时需要再分割成3份嵌入维度的向量
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # 输出投影层
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        self.attn_dropout = nn.Dropout(config.dropout)  # 注意力dropout
        self.resid_dropout = nn.Dropout(config.dropout)  # 残差连接dropout
        self.n_head = config.n_head  # 注意力头数
        self.n_embd = config.n_embd  # 嵌入维度
        self.dropout = config.dropout  # dropout概率
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')  # 检查是否支持flash attention
        
        # 如果不支持flash attention，则注册因果掩码
        # 该因果掩码确保注意力只应用于输入序列中的左侧
        if not self.flash:
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        '''前向传播
        
        参数:
            x: 输入张量 (batch_size, seq_len, n_embd)
        返回:
            注意力输出 (batch_size, seq_len, n_embd)
        '''
        B, T, C = x.size()  # batch_size, seq_len, n_embd
        
        # 计算Q,K,V
        # split函数将c_attn的输出分割成3份嵌入维度的向量
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        
        # 重塑为多头形式 (B, n_head, T, head_dim)
        # 下面多头注意力的计算过程可参考"10.5节 多头注意力"
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        # 使用flash attention或手动实现
        # PyTorch包含scaled_dot_product_attention函数，功能满足要求
        if self.flash:
            '''使用PyTorch的高效scaled_dot_product_attention'''
            y = torch.nn.functional.scaled_dot_product_attention(
                q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            '''手动实现注意力机制'''
            # 计算注意力分数，缩放点积注意力机制，可参考
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            # 应用因果掩码
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            # softmax归一化
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            # 计算注意力输出
            y = att @ v
            
        # 合并多头输出
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        # 投影和dropout
        y = self.resid_dropout(self.c_proj(y))
        return y

- 前馈网络
  - MLP类作为一个前馈网络层的实现。
  - 在Transformer架构中，这样的前馈网络常见于每一个注意力模块之后的位置，并对序列中的每个位置都执行相同的操作。
  - 该层一个完全连接的网络层，通常用于**特征的非线性变换**。

In [5]:
# 前馈网络
class MLP(nn.Module):
    '''多层感知机模块
    
    参数:
        config: GPT配置对象
    '''
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)  # 扩展维度
        self.gelu = nn.GELU()  # GELU激活函数
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)  # 投影回原维度
        self.dropout = nn.Dropout(config.dropout)  # dropout层

    def forward(self, x):
        '''前向传播
        
        参数:
            x: 输入张量
        返回:
            前馈网络输出
        '''
        x = self.c_fc(x)#全连接层
        x = self.gelu(x)#激活层
        x = self.c_proj(x)#投影层
        x = self.dropout(x)
        return x

- Transformer块
  - 构建Decoder块，正如前文所述，和transformer原始解码器的结构有所区别。

In [6]:
# Transformer块
class Block(nn.Module):
    '''Transformer块，包含自注意力和前馈网络
    
    参数:
        config: GPT配置对象
    '''
    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)  # 第一个层归一化
        self.attn = CausalSelfAttention(config)  # 自注意力层
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)  # 第二个层归一化
        self.mlp = MLP(config)  # 前馈网络

    def forward(self, x):
        '''
        前向传播。
        注意：GPT-2将LayerNorm层归一化放在残差连接之前。
        参数:
            x: 输入张量
        返回:
            Transformer块输出
        '''
        # 残差连接 + 自注意力 + 层归一化
        x = x + self.attn(self.ln_1(x))
        # 残差连接 + 前馈网络 + 层归一化
        x = x + self.mlp(self.ln_2(x))
        return x

### 15.8.4.2 GPT实现类
- 包括模型的构建、前向传播、参数初始化、优化器配置以及如何用模型进行文本生成。

In [7]:
# GPT模型
class GPT(nn.Module):
    '''完整的GPT模型实现
    
    参数:
        config: GPT配置对象
    '''
    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None  # 必须指定词汇表大小
        assert config.block_size is not None  # 必须指定块大小
        self.config = config  # 保存配置
        
        # Transformer结构
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),  # 词嵌入
            wpe = nn.Embedding(config.block_size, config.n_embd),  # 位置嵌入
            drop = nn.Dropout(config.dropout),  # dropout层
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),  # Transformer块列表
            ln_f = LayerNorm(config.n_embd, bias=config.bias)  # 最终层归一化
        ))
        # 语言模型头
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # 共享词嵌入和语言模型头的权重，形状皆为：(vocab_size, n_embd)
        self.transformer.wte.weight = self.lm_head.weight

        # 初始化权重
        self.apply(self._init_weights)
        # 特殊初始化投影层
        # GPT-2/GPT-3论文中均提到，对投影层采用缩放的初始化能显著提升深层模型的训练稳定性
        # 注意'/math.sqrt(2 * config.n_layer)'操作
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        '''计算模型参数数量
        
        参数:
            non_embedding: 是否排除位置嵌入参数
        返回:
            参数数量
        '''
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        '''初始化权重
        
        参数:
            module: 要初始化的模块
        '''
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        '''前向传播
        
        参数:
            idx: 输入token索引 (batch_size, seq_len)
            targets: 目标token索引 (batch_size, seq_len)
        返回:
            如果提供targets: (logits, loss)
            否则: (logits, None)
        '''
        device = idx.device
        b, t = idx.size() # 批量大小和序列长度，后者等于block_size参数大小，即最大长度
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device)  # 位置编码

        # 获取词嵌入和位置嵌入
        tok_emb = self.transformer.wte(idx)  # token嵌入 (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos)  # 位置嵌入 (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)  # 相加并应用dropout
        
        # 通过所有Transformer块
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)  # 最终层归一化

        if targets is not None:
            '''训练模式: 计算所有位置的logits和loss'''
            logits = self.lm_head(x)# 输出层
            # view操作后logits形状为(B*T,vocab_size),targets的形状为(B*T,)
            # ignore_index表示填充为-1的元素不参与损失计算
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            '''推理模式: 只计算最后一个位置的logits'''
            logits = self.lm_head(x[:, [-1], :])  # 注意: 使用列表[-1]保持维度
            loss = None

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        '''配置优化器
        
        参数:
            weight_decay: 权重衰减系数
            learning_rate: 学习率
            betas: Adam优化器的beta参数
            device_type: 设备类型('cuda'或'cpu')
        返回:
            配置好的优化器
        '''        
        # 分离可衰减和不可衰减参数
        param_dict = {pn: p for pn, p in self.named_parameters() if p.requires_grad}
        '''
        （1）2D+参数（权重矩阵）：通常是全连接层（nn.Linear）的 weight 或卷积层（nn.Conv2d）的 kernel，
                                形状为 (out_features, in_features) 或 (out_channels, in_channels, kH, kW)。
            需要正则化：权重矩阵容易过拟合，通过权重衰减约束其数值大小（类似L2正则化）。
        （2）1D参数（偏置等）：如 nn.Linear 的 bias 或 LayerNorm 的 weight/bias，形状为 (features,)。
            不应用权重衰减：偏置项对模型输出的平移影响较小，正则化收益低。
                           对LayerNorm的缩放/平移参数正则化可能破坏归一化的稳定性。
         参考：Decoupled Weight Decay Regularization，网址：https://arxiv.org/pdf/1711.05101
        '''
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]  # 只对2D+权重应用weight decay
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]  # 不对偏置等应用weight decay
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        # 使用fused AdamW如果可用
        # 通过内核融合（Kernel Fusion）技术将多个计算步骤合并为单个GPU操作，显著提升训练速度
        # 尤其在大规模深度学习模型中表现突出
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        return torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        '''生成文本
        
        参数:
            idx: 初始token索引 (batch_size, seq_len)
            max_new_tokens: 要生成的最大token数
            temperature: 温度参数(控制随机性)
            top_k: top-k采样参数
        返回:
            生成的token序列
        '''
        for _ in range(max_new_tokens):
            # 如果序列太长，截断到block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # 获取预测,logits形状：(1, t, vocab_size)
            logits, _ = self(idx_cond) 
            # 应用温度，温度值越大，结果越丰富越多样；越小，结果越保守越确定
            # logits变形为[1,vocab_size]
            logits = logits[:, -1, :] / temperature
            # 可选地应用top-k过滤
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                # 注意[-1]的用法，表示维度不变
                logits[logits < v[:, [-1]]] = -float('Inf')
            # 转换为概率
            probs = F.softmax(logits, dim=-1)
            # 采样下一个token，多项式采样方法
            idx_next = torch.multinomial(probs, num_samples=1)
            # 添加到序列中
            # 在推理时，输入是递增的
            #本次的预测输出要增加到已预测的序列末尾作为下一次的输入
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

### 15.8.4.3 数据准备
- 为了最快了解GPT的原理以及实战技巧，karpathy提供了一个训练GPT的莎士比亚作品数据集。
- 首先将其下载为单个(1MB)文件，并将其从原始文本转换为一个大的整数流，即符号级的词元ID。
- 训练参数：
  - 以字符为单位的数据集长度:1115394
  - 所有唯一性字符，即词表中的词元:
    - "\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
  - 字符级词表大小: 65


In [8]:
# 数据准备
def prepare_data():
    '''准备训练数据
    
    返回:
        train_ids: 训练数据token索引
        val_ids: 验证数据token索引
        vocab_size: 词汇表大小
        stoi: 字符到索引的映射
        itos: 索引到字符的映射
    '''
    input_file_path = r'../data/gpt2/tinyshakespeare_input.txt'
    # 如果文件不存在，下载数据
    if not os.path.exists(input_file_path):
        url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
        with open(input_file_path, 'w') as f:
            f.write(requests.get(url).text)
    
    # 读取数据
    with open(input_file_path, 'r') as f:
        data = f.read()
    print(f"Dataset length: {len(data):,} characters")

    # 创建词汇表
    chars = sorted(list(set(data)))
    vocab_size = len(chars)
    print("Unique characters:", ''.join(chars))
    print(f"Vocab size: {vocab_size:,}")

    # 创建字符到索引和索引到字符的映射
    stoi = { ch:i for i,ch in enumerate(chars) }
    itos = { i:ch for i,ch in enumerate(chars) }
    
    # 编码解码函数
    def encode(s): return [stoi[c] for c in s]
    def decode(l): return ''.join([itos[i] for i in l])
    
    # 划分训练集和验证集
    n = len(data)
    train_data = data[:int(n*0.9)]
    val_data = data[int(n*0.9):]
    
    # 编码数据
    train_ids = encode(train_data)
    val_ids = encode(val_data)
    print(f"Train has {len(train_ids):,} tokens")
    print(f"Val has {len(val_ids):,} tokens")
    
    return train_ids, val_ids, vocab_size, stoi, itos

- 数据集的小批量化

In [9]:
# 获取批量数据
def get_batch(split, train_ids, val_ids, batch_size, block_size, device):
    '''获取一批训练或验证数据
    
    参数:
        split: 'train'或'val'
        train_ids: 训练数据
        val_ids: 验证数据
        batch_size: 批量大小
        block_size: 块大小(序列长度)
        device: 设备('cuda'或'cpu')
    返回:
        x: 输入序列 (batch_size, block_size)
        y: 目标序列 (batch_size, block_size)
    '''
    data = train_ids if split == 'train' else val_ids
    # 随机选择起始位置
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # 构造输入和目标
    x = torch.stack([torch.tensor(data[i:i+block_size], dtype=torch.long) for i in ix])
    y = torch.stack([torch.tensor(data[i+1:i+1+block_size], dtype=torch.long) for i in ix])
    return x.to(device), y.to(device)

### 15.8.4.4 训练GPT模型
- 本示例主要是演示GPT模型结构，因此使用karpathy提供的莎士比亚数据进行训练。
- 此微型模型是一个字符级，主要为了便于在PC机上进行训练。

In [10]:
# 训练函数
def train_model():
    '''训练GPT模型
    
    返回:
        model: 训练好的模型
        stoi: 字符到索引的映射
        itos: 索引到字符的映射
    '''
    # 准备数据
    train_ids, val_ids, vocab_size, stoi, itos = prepare_data()
    
    # 配置模型
    config = GPTConfig(
        vocab_size=vocab_size,
        block_size=256,
        n_layer=6,
        n_head=6,
        n_embd=384,
        dropout=0.2,
        bias=False
    )
    
    # 训练参数
    batch_size = 64
    max_iters = 1000
    learning_rate = 1e-3
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtype = 'float16' if torch.cuda.is_available() else 'float32'
    
    # 初始化模型
    model = GPT(config)
    model.to(device)
    
    # 配置优化器
    optimizer = model.configure_optimizers(
        weight_decay=1e-1,
        learning_rate=learning_rate,
        betas=(0.9, 0.95),
        device_type=device
    )
    
    # 训练循环
    best_val_loss = float('inf')
    for iter_num in range(max_iters):
        # 获取批量数据
        xb, yb = get_batch('train', train_ids, val_ids, batch_size, config.block_size, device)
        
        # 前向传播和反向传播
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        
        # 打印训练信息
        if iter_num % 100 == 0:
            print(f"Iter {iter_num}: Train loss {loss.item():.4f}")
    
    # 保存模型
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': config,
        'stoi': stoi,
        'itos': itos
    }, 'gpt_model.pth')
    
    return model, stoi, itos

In [11]:
# 训练模型
model, stoi, itos = train_model()

Dataset length: 1,115,394 characters
Unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65
Train has 1,003,854 tokens
Val has 111,540 tokens
number of parameters: 10.65M
Iter 0: Train loss 4.2706
Iter 100: Train loss 2.7013
Iter 200: Train loss 2.5174
Iter 300: Train loss 2.3994
Iter 400: Train loss 2.2518
Iter 500: Train loss 2.0457
Iter 600: Train loss 1.8641
Iter 700: Train loss 1.7476
Iter 800: Train loss 1.6565
Iter 900: Train loss 1.5957


### 15.8.4.5 GPT文本生成

In [12]:
# 生成文本函数
def generate_text(model, stoi, itos, start="\n", max_new_tokens=2000, temperature=0.8, top_k=200):
    '''使用训练好的模型生成文本
    
    参数:
        model: 训练好的模型
        stoi: 字符到索引的映射
        itos: 索引到字符的映射
        start: 起始字符串
        max_new_tokens: 要生成的最大token数
        temperature: 温度参数
        top_k: top-k采样参数
    '''
    device = next(model.parameters()).device
    
    # 编码解码函数
    def encode(s): return [stoi[c] for c in s]
    def decode(l): return ''.join([itos[i] for i in l])
    
    # 编码起始字符串
    start_ids = encode(start)
    x = torch.tensor(start_ids, dtype=torch.long, device=device).unsqueeze(0)
    
    # 生成文本
    with torch.no_grad():
        y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
        print(decode(y[0].tolist()))

In [13]:
# 生成文本
generate_text(model, stoi, itos)


Prove:
Nay! O, be Joy, what merday, I am in your train
The hastings anger to the stand a enye of.

MENILIUS:
And abso this to rue of you longue
In excut of them aborniship so, and for true, kil bounds?

POMPERIZA:
I thank to thine we'ell neever is these drays, the
cannot heir loal mornish' be thou sad: hare live move
I have a gone?
And masters we to did 'tis follows me.

First I Murderemy, you and the tir king.

Calllent father; if Cous!

HENRY BOLIF GAREY:
Pray Grand the lands me, forth, to give in the pall,
fie may right and should of Rome,
In the world that I selft servil.

KING RICHARD III:
Fir, a must shall were stay,
To the that, they not shate father allones
As aid nothing shall be peersent?

POLIXENESES:
Proud will not mades,
I scoldier of shall had do: who the clannntay;
And with that thy liker of on, who bacwds denign fare you:
I dust be go did it you.

KING RICHARD IING I:
My gorne wisers dalep son:
The city, knows, have in liboses our subject most demight.
I now, you show 

- **参考资料**
  - [Improving Language Understanding by Generative Pre-Training](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)
  - [OpenAI ChatGPT（二）：十分钟读懂 GPT-1](https://zhuanlan.zhihu.com/p/604625917)
  - [ChatGPT/InstructGPT详解](https://zhuanlan.zhihu.com/p/590311003)
  - [GPT-4核心技术探秘](https://zhuanlan.zhihu.com/p/626463196)
  - [图解GPT-2 | The Illustrated GPT-2](https://lolitasian.blog.csdn.net/article/details/125529598)(强烈推荐)
  - [GPT-4官方技术报告](https://cdn.openai.com/papers/gpt-4.pdf)

## 小结
- GPT与ChatGPT的核心架构。GPT系列模型基于Transformer架构，通过自注意力机制处理长距离依赖关系，实现强大的文本生成能力。ChatGPT作为GPT的优化版本，专注于对话场景，通过改进训练技术和人类反馈强化学习（RLHF）提升交互质量。  
- GPT关键训练技术。ChatGPT的训练过程结合了**人类反馈强化学习（RLHF）**和**TAMER框架**，通过人类偏好数据微调模型，使其输出更符合用户需求。GPT-4进一步优化了训练规模、数据多样性及多模态能力（如支持图像输入）。  
- 实现流程与文本生成。实现GPT模型需经历**数据准备**（大规模文本清洗）、**模型构建**（Transformer堆叠）、**训练**（无监督预训练+有监督微调）和**文本生成**（基于概率采样的解码策略）四个阶段，最终生成连贯、上下文相关的文本。